In [1]:
import pdb
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import distance_metrics
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.linalg import svds as SVDS
import pandas as pd
import numpy as np

STOP_WORDS = stopwords.words('english')


In [22]:
with open('./pubmed_vectorizer.pkl', 'rb') as f:
    pubmed_vectorizer = pickle.load(f)

with open('./article_file_titles_list.pkl', 'rb') as f:
    PUBMED_ARTICLE_TITLES = pickle.load(f)

with open('./pubmed_tfidf.pkl', 'rb') as f:
    pubmed_tfidf = pickle.load(f)
pubmed_tfidf = pubmed_tfidf.T

IOError: [Errno 2] No such file or directory: './pubmed_vectorizer.pkl'

if you  need to load pickles

In [ ]:
# count_vectorizer = load_pickle('count_vectorizer.pkl')
# pubmed_vecs = load_pickle('pubmed_vecs.pkl')
# pubmed_lsi_corpus = load_pickle('pubmed_lsi_corpus.pkl')

# Make TFIDF

In [4]:
import pdb 
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models, similarities, matutils, interfaces


DATA_PATH = ('./pmc_data/pmc_text_files/')
GENRE_FOLDERS = os.listdir(DATA_PATH)
ARTICLE_FILE_PATHS = []
ARTICLE_FILE_TITLES = []
ARTICLE_DOCUMENT_LIST = []



def load_articles():
    genre_folders_left = len(GENRE_FOLDERS)
    completed_genre_folders = 0 
    for genre_folder in GENRE_FOLDERS:
        completed_genre_folders += 1
        genre_folder_path = os.path.join(DATA_PATH, genre_folder)
        genre_file_list = os.listdir(genre_folder_path)
        for article_file_title in genre_file_list:
            article_file_path = os.path.join(genre_folder_path, article_file_title)
            if os.path.isfile(article_file_path):
                ARTICLE_FILE_TITLES.append(article_file_title)
                ARTICLE_FILE_PATHS.append(article_file_path)
                with open(article_file_path, 'rb') as f:
                    document = f.read()
                    ARTICLE_DOCUMENT_LIST.append(document)

                print "done with: ", article_file_title
                print "progress: ", completed_genre_folders / float(genre_folders_left)
            else:
                sub_article_folder_list = os.listdir(article_file_path)
                for sub_article_file_title in sub_article_folder_list:
                    sub_article_file_path = os.path.join(article_file_path, 
                            sub_article_file_title)
                    ARTICLE_FILE_TITLES.append(sub_article_file_title)
                    ARTICLE_FILE_PATHS.append(sub_article_file_path)
                    with open(sub_article_file_path, 'rb') as f:
                        document = f.read()
                        ARTICLE_DOCUMENT_LIST.append(document)
                        
load_articles()

done with:  Arch_Dis_Child_2007_Apr_11_92(4)_298-303.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_Aug_18_93(8)_654-659.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_Jul_1_93(7)_566-569.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_Jul_1_93(7)_620-625.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_May_17_93(5)_373-378.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_Oct_2_93(10)_845-850.txt
progress:  0.5
done with:  Arch_Dis_Child_2008_Oct_30_93(10)_890-898.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Feb_14_94(2)_110-116.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Feb_19_94(2)_156-160.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Jan_25_94(1)_11-15.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Jan_9_94(1)_42-46.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Nov_3_94(11)_831-833.txt
progress:  0.5
done with:  Arch_Dis_Child_2009_Oct_29_95(6)_414-421.txt
progress:  0.5
done with:  Arch_Dis_Child_2010_Aug_12_96(1)_30-37.txt
progress:  0.5


## Make corpus!

##### CountVectorize

In [25]:
# count_vectorizer = CountVectorizer(stop_words='english')
# count_vectorizer.fit(ARTICLE_DOCUMENT_LIST)
# pubmed_vecs = count_vectorizer.transform(ARTICLE_DOCUMENT_LIST).transpose()


In [ ]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.iteritems())
pubmed_corpus = matutils.Sparse2Corpus(pubmed_vecs)

##### TFIDF

In [26]:
pubmed_tfidf = models.TfidfModel(pubmed_corpus)

In [27]:
pubmed_tfidf_corpus = pubmed_tfidf[pubmed_corpus]

##### SVD

In [28]:
lsi = models.LsiModel(pubmed_corpus, id2word=id2word, num_topics=25)

In [29]:
pubmed_lsi_corpus = lsi[pubmed_tfidf_corpus]

Which ones have `.save` methods

In [30]:
# pubmed_corpus.save
# pubmed_tfidf_corpus.save
# lsi.save
# pubmed_lsi_corpus.save

AttributeError: 'CountVectorizer' object has no attribute 'sav'

##### Concept similarity between 1st and the rest of documents

In [7]:
pubmed_doc_vecs = [doc for doc in pubmed_lsi_corpus]
index = similarities.MatrixSimilarity(pubmed_doc_vecs)
sims = sorted(enumerate(index[pubmed_doc_vecs[0]]), key=lambda item: -item[1])
sims

[(0, 1.0),
 (3, 0.68401217),
 (91, 0.54140764),
 (6, 0.49282286),
 (69, 0.49231014),
 (100, 0.46222082),
 (132, 0.44177955),
 (70, 0.42240763),
 (104, 0.41264719),
 (60, 0.39334083),
 (108, 0.38543543),
 (39, 0.38287643),
 (107, 0.37517354),
 (2, 0.35641479),
 (4, 0.35301393),
 (19, 0.35044608),
 (111, 0.33500758),
 (16, 0.33337039),
 (51, 0.33177891),
 (84, 0.32585904),
 (117, 0.30051011),
 (53, 0.29050332),
 (90, 0.26644206),
 (65, 0.25168028),
 (110, 0.24598333),
 (130, 0.24549709),
 (125, 0.2340491),
 (62, 0.23323879),
 (88, 0.22533779),
 (113, 0.22299711),
 (92, 0.2027103),
 (11, 0.20220001),
 (21, 0.18942399),
 (34, 0.18222679),
 (105, 0.1800033),
 (63, 0.15380761),
 (97, 0.13903588),
 (122, 0.12866879),
 (17, 0.125273),
 (31, 0.12374873),
 (78, 0.11779362),
 (25, 0.11410177),
 (96, 0.1114743),
 (131, 0.11126879),
 (121, 0.10021187),
 (80, 0.10004397),
 (129, 0.093253285),
 (18, 0.092191935),
 (23, 0.09029454),
 (74, 0.089614481),
 (101, 0.084233455),
 (35, 0.079859219),
 (114, 0

## Fold in a new document

##### Count vectorize the document

In [34]:
article = ["If a patient is ill and takes drugs for that illness, these drugs often lead to further illnesses and complications. This affects millions of people throughout the world. An international research team from Bielefeld University, universities in Novosibirsk (Russia) and Tomsk (Russia), and from the National Academy of Sciences of Ukraine in Kiev are studying how to prevent this problem. Starting in May, the Volkswagen Foundation will be funding this cooperation with about 235,000 Euro for the next two years. The project will be coordinated at Bielefeld University by Professor Dr. Ralf from the Faculty of Technology."]
# article = sent_tokenize(article)
# count_vectorizer.fit(article)
article_vecs = count_vectorizer.transform(article).transpose()
article_vecs_sparse = matutils.Sparse2Corpus(article_vecs)
# article_vecs.shape

In [35]:
article_vecs.shape

(34714, 1)

##### TFIDF

In [36]:
article_tfidf = pubmed_tfidf[article_vecs_sparse]

##### SVD

how's this shape compare to our pubmed_lsi_corpus space? 

In [42]:
article_lsi = lsi[article_tfidf]

In [43]:
article_doc_vecs = [doc for doc in article_lsi]

##### Concept Similarity Between Documents

In [ ]:
index = similarities.MatrixSimilarity(pubmed_doc_vecs)
sims = sorted(enumerate(index[pubmed_doc_vecs[0]]), key=lambda item: -item[1])
sims

In [3]:
ARTICLE_DOCUMENT_LIST[0]

"\n==== Front\nArch Dis ChildArch. Dis. ChildarchdischildadcArchives of Disease in Childhood0003-98881468-2044BMJ Publishing Group BMA House, Tavistock Square, London, WC1H 9JR 26342094archdischild-2015-30872310.1136/archdischild-2015-3087231506Original ArticleAdmission to hospital for bronchiolitis in England: trends over five decades, geographical variation and association with perinatal characteristics and subsequent asthma Green Christopher A 1Yeates David 2Goldacre Allie 2Sande Charles 1Parslow Roger C 3McShane Philip 3Pollard Andrew J 1Goldacre Michael J 21 Department of Paediatrics, University of Oxford, and the NIHR Oxford Biomedical Research Centre, UK2 Unit of Health-Care Epidemiology, Nuffield Department of Population Health, University of Oxford, Oxford, UK3 Division of Epidemiology and Biostatistics, University of Leeds, Leeds, UKCorrespondence to  Dr Christopher A Green, Oxford Vaccine Group, Centre for Clinical Vaccinology & Tropical Medicine, Churchill Hospital, Oxford 

In [10]:
model = models.tfidfmodel.TfidfModel(ARTICLE_DOCUMENT_LIST)

ValueError: need more than 1 value to unpack